<a href="https://colab.research.google.com/github/jonnells/cmpe255-project-grp/blob/main/Yelp_On_The_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yelp On the Go!

In [1]:
import os
import json

## Import Data

In [3]:
yelp_dataset_path = 'yelp_dataset/'

In [4]:
user_data = []
with open(yelp_dataset_path + 'yelp_academic_dataset_user.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_user.json')
    for line in fin:
        user_data.append(json.loads(line))

Reading yelp_academic_dataset_user.json


In [5]:
review_data = []
with open(yelp_dataset_path + 'yelp_academic_dataset_review.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_review.json')
    for line in fin:
        review_data.append(json.loads(line))

Reading yelp_academic_dataset_review.json


In [15]:
business_data = []
with open(yelp_dataset_path + 'yelp_academic_dataset_business.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_business.json')
    for line in fin:
        business_data.append(json.loads(line))

Reading yelp_academic_dataset_business.json


## Data Investigation

In [52]:
print(str(len(user_data)) + ' user records loaded')
print(str(len(review_data)) + ' review records loaded')
print(str(len(business_data)) + ' business records loaded')

726250 user records loaded
8635403 review records loaded
160470 business records loaded


In [10]:
print(user_data[0].keys())

dict_keys(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos'])


In [11]:
print(review_data[0].keys())

dict_keys(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])


In [16]:
print(business_data[0].keys())

dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])


## Data Pre-processing

Remove users that do not have enough reviews (10)

In [76]:
user_data = [user for user in user_data if user.get('review_count') >= 15]

In [77]:
print(str(len(user_data)) + ' useful user records')

431741 useful user records


Only keep restaurant related businesses

In [62]:
restaurant_data = [business for business in business_data if restaurant_category in business.get('categories')]

In [79]:
print(str(len(restaurant_data)) + ' restaurant records')

50763 restaurant records


Grab imporant attributes

In [81]:
restaurant_ids = [restaurant.get('business_id') for restaurant in restaurant_data]

In [86]:
restaurant_stars = [restaurant.get('stars') for restaurant in restaurant_data]

In [88]:
user_ids = [user.get('user_id') for user in user_data]